# LangChain MongoDB Integration - Implement RAG Locally

This notebook is a companion to the [LangChain Local RAG](https://www.mongodb.com/docs/atlas/atlas-vector-search/ai-integrations/langchain/get-started/) tutorial. Refer to the page for set-up instructions and detailed explanations.

<a target="_blank" href="https://colab.research.google.com/github/mongodb/docs-notebooks/blob/main/ai-integrations/langchain-local-rag.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Create a local Atlas deployment

Run the following commands in your terminal to set up your local Atlas deployment. 

```
atlas deployments setup
curl  https://atlas-education.s3.amazonaws.com/sampledata.archive -o sampledata.archive
mongorestore --archive=sampledata.archive --port=<port-number>
```

## Set up the environment

In [ ]:
pip install --quiet --upgrade pymongo langchain gpt4all sentence_transformers langchain-huggingface

In [ ]:
MONGODB_URI = ("mongodb://localhost:64983/?directConnection=true")

## Create embeddings with local model

In [ ]:
from pymongo import MongoClient
from sentence_transformers import SentenceTransformer

# Connect to your local Atlas deployment or Atlas Cluster
client = MongoClient(MONGODB_URI)

# Select the sample_airbnb.listingsAndReviews collection
collection = client["sample_airbnb"]["listingsAndReviews"]

# Load the embedding model (https://huggingface.co/mixedbread-ai/mxbai-embed-large-v1)
model_path = "/Users/david.hou/Desktop/work/docs-notebooks/testing"
model = SentenceTransformer('mixedbread-ai/mxbai-embed-large-v1')
model.save(model_path)
model = SentenceTransformer(model_path)

# Define function to generate embeddings
def get_embedding(text):
    return model.encode(text).tolist()

# Filters for only documents with a summary field and without an embeddings field
filter = { '$and': [ { 'summary': { '$exists': True, "$nin": [ None, "" ] } }, { 'embeddings': { '$exists': False } } ] }

# Creates embeddings for subset of the collection
updated_doc_count = 0
for document in collection.find(filter).limit(50):
    text = document['summary']
    embedding = get_embedding(text)
    collection.update_one({ '_id': document['_id'] }, { "$set": { 'embeddings': embedding } }, upsert=True)
    updated_doc_count += 1

print("Documents updated: {}".format(updated_doc_count))


## Configure the vector store

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="mixedbread-ai/mxbai-embed-large-v1")

from langchain_mongodb import MongoDBAtlasVectorSearch
# Instantiate vector store
vector_store = MongoDBAtlasVectorSearch.from_connection_string(
   connection_string = MONGODB_URI,
   namespace = "sample_airbnb.listingsAndReviews",
   embedding=embedding_model,
   index_name="vector_index",
   embedding_key="embeddings",
   text_key="summary")

In [ ]:
vector_store.create_vector_search_index(
  dimensions = 1024,       # The dimensions of the vector embeddings to be indexed
  wait_until_complete = 60 # Number of seconds to wait for the index to build (can take around a minute)
)

## Implement RAG with a local LLM
Before running the following code, [download the local model](https://gpt4all.io/models/gguf/mistral-7b-openorca.gguf2.Q4_0.gguf).

In [ ]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.llms import GPT4All

# Configure the LLM
local_path = "<path-to-model>"

# Callbacks support token-wise streaming
callbacks = [StreamingStdOutCallbackHandler()]

# Verbose is required to pass to the callback manager
llm = GPT4All(model=local_path, callbacks=callbacks, verbose=True)

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import pprint

# Instantiate Atlas Vector Search as a retriever
retriever = vector_store.as_retriever()

# Define prompt template
template = """
Use the following pieces of context to answer the question at the end.
{context}
Question: {question}
"""
custom_rag_prompt = PromptTemplate.from_template(template)

def format_docs(docs):
   return "\n\n".join(doc.page_content for doc in docs)

# Create chain   
rag_chain = (
   {"context": retriever | format_docs, "question": RunnablePassthrough()}
   | custom_rag_prompt
   | llm
   | StrOutputParser()
)

# Prompt the chain
question = "Can you recommend me a few AirBnBs that are beach houses?"
answer = rag_chain.invoke(question)

# Return source documents
documents = retriever.invoke(question)
print("\nSource documents:")
pprint.pprint(documents)